## Introduction
Modern geochemical and environmental datasets—especially those generated by ICP‑MS, ICP‑OES, or XRF instruments—require rigorous Quality Control (QC) before any interpretation or modelling can begin. Analytical workflows often include thousands of samples and dozens of analytes, along with standards, blanks, duplicates, and censored (below‑detection‑limit) values.

This notebook builds a lightweight, transparent, and fully reproducible QC toolkit in Python, designed to mirror the core checks used in professional laboratories and mining operations. The goal is to demonstrate how QC logic can be automated using clean, readable code that scales to multi‑element datasets.

The notebook covers:

- Detection limits & censored data handling  
Configurable substitution methods (DL/2, DL/√2, zero).

- Standards QC  
%Bias calculations and pass/fail evaluation against certified values.

- Blanks QC  
Contamination checks using configurable thresholds (e.g., 5× DL).

- Z‑scores & Shewhart‑type control rules  
Simple drift detection for monitoring instrument stability.

- QC summary functions  
A unified structure for flagging precision, accuracy, contamination, and drift.

Each function is modular and easy to integrate into larger pipelines, making this notebook a practical foundation for real‑world assay validation, exploration geochemistry, or environmental monitoring.

#### Using Dictionaries for Geochemical Assay Data
Python dictionaries are a convenient way to store assay results by element, especially when working with multi‑element geochemical datasets. Each key represents an element (and its unit), and each value stores the corresponding assay result.

Dictionaries allow us to:

- Access any element’s assay quickly
- Keep units visible and explicit (e.g., ppm, %, ppb)
- Organize multi‑element data in a clean, readable structure
- Pass assay sets into QC or geochemical functions easily

In [1]:
# Dictionary storing assay values for multiple elements
assay = {"Au_ppm": 1.25, "Ag_ppm": 3.1, "Cu_pct": 0.42}

# Accessing the gold value
print(assay["Au_ppm"])


1.25


Assay datasets often contain dozens of elements.

Dictionaries allow us to reference each element by name, rather than relying on column positions.
 
This structure integrates naturally with QC functions such as:

- RPD for duplicates
- Detection limit checks
- Standard and blank QC
- ALR/CLR transformations
- Interval compositing
- This makes dictionaries a simple but powerful building block for building a geochemical QC toolkit in Python.

#### List Comprehensions in Python (Geochemistry Example)
A list comprehension is a compact and readable way to create a new list by transforming the elements of an existing list.
It replaces longer for‑loops with a single clean expression.

This is especially useful in geochemistry when converting units or applying the same transformation to many assay values.

##### Example: Converting ppm to percent

In [2]:
ppm = [0.2, 0.5, 1.0]
percent = [x / 10000 for x in ppm]  # 1% = 10,000 ppm
percent


[2e-05, 5e-05, 0.0001]

ppm is a list of values in parts per million.

The list comprehension [x / 10000 for x in ppm] means:

1- Take each value x in the list ppm

2- Divide it by 10,000 to convert ppm → percent

3- store the result in a new list called percent

#### Handling Exceptions in Python: Safe Division Function
In real geochemical workflows, division by zero can happen easily—for example when calculating ratios, enrichment factors, or percent differences where the denominator might be zero or below detection limit.

A clean way to prevent your code from crashing is to use exception handling.

The function below demonstrates a simple and practical pattern:

In [3]:
def safe_div(a: float, b: float) -> float:
    try:
        return a / b
    except ZeroDivisionError:
        return float("nan")

- It attempts to divide a by b.
- If b is zero, Python would normally raise a ZeroDivisionError.
- The try/except block catches that error and returns NaN instead of stopping the program.

This pattern is useful when:

- Calculating RPD where (a + b) might be zero
- Computing ratios (e.g., Cu/Zn, As/Sb)
- Working with detection limits where values may be zero
- Running QC pipelines where you want the script to continue even if some values are invalid
- Returning NaN keeps the workflow stable and allows downstream functions (pandas, numpy, plotting) to handle missing or invalid values gracefully.

#### Geochemistry Helper Functions — Unit Conversions (ppm ↔ % ↔ g/t)
Unit conversions are a routine part of geochemical data processing. Assay datasets often mix units such as ppm, percent, and g/t, and converting between them cleanly is essential for QC, reporting, and downstream calculations.

The functions below provide simple, readable conversions commonly used in exploration and mining geochemistry.

##### 1. ppm → percent
10,000 ppm = 1%  
##### 1% = 1/100 = 10,000 ppm



In [4]:
def ppm_to_percent(ppm: float) -> float:
    """Convert ppm to percent. 10,000 ppm = 1%."""
    return ppm / 10000


##### 2. percent → ppm
The inverse conversion:
##### ppm = percent × 10,000

In [5]:
def percent_to_ppm(percent: float) -> float:
    """Convert percent to ppm."""
    return percent * 10000


##### 3. ppm → g/t (for Au/Ag)
For solid rock samples, 1 ppm ≈ approximately equals 1 g/t, especially for precious metals such as gold and silver.
- ppm = mg/kg
- g/t = g per tonne
- 1 mg/kg = 1 g/t
- So the conversion is effectively an identity.

In [6]:
def ppm_to_gpt(ppm: float) -> float:
    """For Au/Ag: ppm ≈ g/t in solids (1 ppm ~ 1 g/t)."""
    return ppm  # identity


In [7]:
print(ppm_to_percent(4200))    
print(percent_to_ppm(0.42))    
print(ppm_to_gpt(1.5))         


0.42
4200.0
1.5


Usage:
- ICP-MS and ICP-OES outputs often mix ppm and percent
- Precious metals (Au, Ag) are typically reported in g/t
- Base metals (Cu, Pb, Zn) may appear in % or ppm depending on grade
- QC checks (standards, duplicates, blanks) require consistent units
- Compositing and metal-content calculations depend on correct units

#### Relative Percent Difference (RPD) — Precision Check for Duplicate Assays
The Relative Percent Difference (RPD) is one of the most widely used metrics in geochemical QA/QC for evaluating precision between duplicate samples.
It measures how closely two assay values agree with each other, expressed as a percentage.

A low Relative Percent Difference- RPD- means the duplicate pair is consistent; a high RPD suggests poor precision or potential analytical issues.
$$
RPD = \frac{|a - b|}{\frac{a + b}{2}} \times 100
$$

- 𝑎 = primary assay
- 𝑏 = duplicate assay
- The denominator uses the mean of the two values to normalize the difference.

In [8]:
def rpd(a: float, b: float) -> float:
    """Relative Percent Difference: abs(a - b) / ((a + b)/2) * 100."""
    if (a + b) == 0:
        return 0.0
    return abs(a - b) / ((a + b) / 2) * 100

# Examples
print(rpd(1.25, 1.30)) 
print(rpd(0.05, 0.05))  


3.921568627450984
0.0


 if we want to automatically detect red flags in your QC workflow, use a clean, reusable Python function that checks:

- High RPD (precision failure)
- High %Bias (accuracy failure)
- High blank values (contamination)

Any custom thresholds we define, if we want to automatically detect red flags in your QC workflow: 
- High RPD (precision failure)
- High %Bias (accuracy failure)
- High blank values (contamination)
- Any custom thresholds you define

#### FLAG QC

In [9]:
def qc_flag(
    rpd_value: float = None,
    bias_value: float = None,
    blank_value: float = None,
    rpd_limit: float = 20,
    bias_limit: float = 10,
    blank_limit: float = 0.005
) -> str:
    """
    Return a QC flag based on common geochemical QA/QC thresholds.

    Parameters
    ----------
    rpd_value : float
        Relative Percent Difference between duplicates.
    bias_value : float
        Percent bias for standards.
    blank_value : float
        Assay value for blank samples.
    rpd_limit : float
        Maximum acceptable RPD (%).
    bias_limit : float
        Maximum acceptable %Bias (%).
    blank_limit : float
        Maximum acceptable blank value (same units as assay).

    Returns
    -------
    str
        QC flag: "PASS", "FAIL_PRECISION", "FAIL_ACCURACY",
        "FAIL_BLANK", or "MULTIPLE_FAILURES".
    """

    flags = []   # starts an empty list that will collect any QC problems found for that sample.

    # Precision check
    if rpd_value is not None and rpd_value > rpd_limit:
        flags.append("FAIL_PRECISION")

    # Accuracy check
    if bias_value is not None and abs(bias_value) > bias_limit:
        flags.append("FAIL_ACCURACY")

    # Contamination check
    if blank_value is not None and blank_value > blank_limit:
        flags.append("FAIL_BLANK")

    # Final decision
    if len(flags) == 0:
        return "PASS"
    if len(flags) == 1:
        return flags[0]
    return "MULTIPLE_FAILURES"


In [10]:
print(qc_flag(rpd_value=25)) # FAIL_PRECISION 
print(qc_flag(bias_value=12)) # FAIL_ACCURACY 
print(qc_flag(blank_value=0.02)) # FAIL_BLANK 
print(qc_flag(rpd_value=30, bias_value=15)) # MULTIPLE_FAILURES

FAIL_PRECISION
FAIL_ACCURACY
FAIL_BLANK
MULTIPLE_FAILURES


#### QC Summary Table

In [11]:
import pandas as pd

# ---------------------------------------------------------
# QC SUMMARY FUNCTION
# ---------------------------------------------------------
def qc_summary(
    sample_id,
    rpd_value=None,
    bias_value=None,
    zscore_value=None,
    blank_value=None,
    rpd_limit=20,
    bias_limit=10,
    z_limit=3,
    blank_limit=0.005
):
    flags = []

    if rpd_value is not None and rpd_value > rpd_limit:
        flags.append("FAIL_PRECISION")

    if bias_value is not None and abs(bias_value) > bias_limit:
        flags.append("FAIL_ACCURACY")

    if zscore_value is not None and abs(zscore_value) > z_limit:
        flags.append("FAIL_ZSCORE")

    if blank_value is not None and blank_value > blank_limit:
        flags.append("FAIL_BLANK")

    if len(flags) == 0:
        final_flag = "PASS"
    elif len(flags) == 1:
        final_flag = flags[0]
    else:
        final_flag = "MULTIPLE_FAILURES"

    row = {
        "SampleID": sample_id,
        "RPD": rpd_value,
        "%Bias": bias_value,
        "Zscore": zscore_value,
        "BlankValue": blank_value,
        "QC_Flag": final_flag
    }

    return pd.DataFrame([row])


# ---------------------------------------------------------
# EXAMPLE QC ROWS
# ---------------------------------------------------------
df1 = qc_summary("STD-01", rpd_value=None, bias_value=12.5, zscore_value=3.8)
df2 = qc_summary("DUP-15", rpd_value=28.0)
df3 = qc_summary("BLK-03", blank_value=0.02)
df4 = qc_summary("SMP-100", rpd_value=5.2, bias_value=2.1, blank_value=0.0001)


# ---------------------------------------------------------
# FIX: DROP ALL-NaN COLUMNS BEFORE CONCAT
# ---------------------------------------------------------
clean_dfs = []
for df in [df1, df2, df3, df4]:
    df_clean = df.dropna(axis=1, how="all")   # remove all-NaN columns
    clean_dfs.append(df_clean)

qc_table = pd.concat(clean_dfs, ignore_index=True)

qc_table


,SampleID,%Bias,Zscore,QC_Flag,RPD,BlankValue
0,STD-01,12.5,3.8,MULTIPLE_FAILURES,NaN,NaN
1,DUP-15,NaN,NaN,FAIL_PRECISION,28.0,NaN
2,BLK-03,NaN,NaN,FAIL_BLANK,NaN,0.0200
3,SMP-100,2.1,NaN,PASS,5.2,0.0001


In [12]:
import pandas as pd

# ---------------------------------------------------------
# QC SUMMARY FUNCTION
# ---------------------------------------------------------
def qc_summary(
    sample_id,
    rpd_value=None,
    bias_value=None,
    zscore_value=None,
    blank_value=None,
    rpd_limit=20,
    bias_limit=10,
    z_limit=3,
    blank_limit=0.005
):
    flags = []

    if rpd_value is not None and rpd_value > rpd_limit:
        flags.append("FAIL_PRECISION")

    if bias_value is not None and abs(bias_value) > bias_limit:
        flags.append("FAIL_ACCURACY")

    if zscore_value is not None and abs(zscore_value) > z_limit:
        flags.append("FAIL_ZSCORE")

    if blank_value is not None and blank_value > blank_limit:
        flags.append("FAIL_BLANK")

    if len(flags) == 0:
        final_flag = "PASS"
    elif len(flags) == 1:
        final_flag = flags[0]
    else:
        final_flag = "MULTIPLE_FAILURES"

    row = {
        "SampleID": sample_id,
        "RPD": rpd_value,
        "%Bias": bias_value,
        "Zscore": zscore_value,
        "BlankValue": blank_value,
        "QC_Flag": final_flag
    }

    return pd.DataFrame([row])


# ---------------------------------------------------------
# EXAMPLE QC ROWS
# ---------------------------------------------------------
df1 = qc_summary("STD-01", rpd_value=None, bias_value=12.5, zscore_value=3.8)
df2 = qc_summary("DUP-15", rpd_value=28.0)
df3 = qc_summary("BLK-03", blank_value=0.02)
df4 = qc_summary("SMP-100", rpd_value=5.2, bias_value=2.1, blank_value=0.0001)


# ---------------------------------------------------------
# FIX: DROP ALL-NaN COLUMNS BEFORE CONCAT
# ---------------------------------------------------------
clean_dfs = []
for df in [df1, df2, df3, df4]:
    df_clean = df.dropna(axis=1, how="all")   # remove all-NaN columns
    clean_dfs.append(df_clean)

qc_table = pd.concat(clean_dfs, ignore_index=True)

qc_table


,SampleID,%Bias,Zscore,QC_Flag,RPD,BlankValue
0,STD-01,12.5,3.8,MULTIPLE_FAILURES,NaN,NaN
1,DUP-15,NaN,NaN,FAIL_PRECISION,28.0,NaN
2,BLK-03,NaN,NaN,FAIL_BLANK,NaN,0.0200
3,SMP-100,2.1,NaN,PASS,5.2,0.0001


### 1. QC Summary Function
The qc_summary() function creates a single‑row DataFrame for any QC item.
It evaluates several common QA/QC checks:

- RPD → precision (duplicates)
- %Bias → accuracy (standards)
- Z‑score → accuracy (standards)
- Blank value → contamination (blanks)

Based on these values, the function assigns a final QC flag:

- "PASS"
- "FAIL_PRECISION"
- "FAIL_ACCURACY"
- "FAIL_ZSCORE"
- "FAIL_BLANK"
- "MULTIPLE_FAILURES"

The function returns a tidy one‑row DataFrame containing all metrics and the final QC decision.

### 2. Creating Example QC Rows
We generate four example QC entries:

- A standard with %Bias and Z‑score
- A duplicate with RPD
- A blank with a blank value
- A regular sample with several QC metrics

Each call to qc_summary() returns a small DataFrame representing one QC item.

### 3. Cleaning QC Rows Before Combining
Different QC types contain different fields.
For example:

- Duplicates do not have %Bias or Z‑score
- Blanks do not have RPD
- Standards do not have blank values

This means some columns are empty (all NaN) for certain rows.

Before combining the rows, we remove any columns that are completely empty:

`df_clean = df.dropna(axis=1, how="all")`

### 4. Building the Final QC Table
All cleaned QC rows are collected into a list and concatenated:

`qc_table = pd.concat(clean_dfs, ignore_index=True)`

The result is a compact, readable QC summary table that shows:

- Sample ID
- Available QC metrics
- Final QC flag

This table is suitable for reporting, dashboards, or further analysis.

#### Detection Limits (DL) and How to Handle Censored Data
Analytical instruments such as ICP‑MS, ICP‑OES, or XRF can only `measure concentrations down to a certain minimum level`.
**Anything below that threshold is considered Below Detection Limit (BDL) or censored data**.

A **Detection Limit (DL)** `is the lowest concentration an instrument can reliably distinguish from zero.`
If a true concentration is lower than this limit, the instrument cannot measure it accurately, so the reported value is:

- None
- 0
- "<DL"
- or a very small number that is not meaningful

These values cannot be used directly in statistical analysis, so we replace them with a substitution value.

**Common Substitution Methods for BDL Values**
There are two widely used approaches in geochemistry and environmental science:

1. **DL/2 (Half the detection limit)**
A simple, conservative substitution.
Useful when the proportion of censored data is small.

2. **DL/√2 (DL divided by 1.4142)**
A statistically motivated substitution based on log‑normal assumptions.
Often used in environmental datasets.

3. **Zero substitution**
Not recommended for most geochemical work, but sometimes used in machine‑learning preprocessing.

The function makes this choice configurable, which is exactly what a good QC toolkit should do.

##### Python Function: Substitute Below‑Detection‑Limit Values

In [13]:
def substitute_bdl(value: float | None, dl: float, method: str = "half") -> float:
    """
    Substitute below-detection-limit values.
    method: 'half' -> DL/2, 'sqrt2' -> DL / 1.4142, 'zero' -> 0
    """
    if value is None or value < dl:
        if method == "half":
            return dl / 2
        elif method == "sqrt2":
            return dl / 1.41421356237
        elif method == "zero":
            return 0.0
        else:
            raise ValueError("method must be 'half', 'sqrt2', or 'zero'")
    return value


This snipped code takes a measured value and the detection limit (dl). Checks whether the value is:

- missing (None)
- or below the detection limit

If so, replace it using the chosen method:

- "half" → DL/2
- "sqrt2" → DL/√2
- "zero" → 0

If the value is above DL, it is returned unchanged.

Example:

In [14]:
print(substitute_bdl(0.0003, dl=0.001, method="half"))

print(substitute_bdl(None, 0.005, method="sqrt2"))


0.0005
0.003535533905940475


### Standards & Blanks QC
Quality Control (QC) in geochemical and environmental analysis relies heavily on two special types of samples:

1. `Standards` (Certified Reference Materials – CRM) -->
Standards have known, certified concentrations for each analyte.
They are used to check accuracy of the instrument.

A standard passes QC when:

**%𝐵𝑖𝑎𝑠=𝑀𝑒𝑎𝑠𝑢𝑟𝑒𝑑−𝐶𝑒𝑟𝑡𝑖𝑓𝑖𝑒𝑑 / 𝐶𝑒𝑟𝑡𝑖𝑓𝑖𝑒𝑑 × 100**

is within an acceptable tolerance (commonly ±10–20%).

2. `Blanks` -->
Blanks contain no analytes and are used to check contamination.

A blank fails QC if the measured value exceeds a contamination threshold, often defined as:

**Threshold = 5 × Detection Limit**

This ensures that any carryover or contamination is detected early.

#### 1. Percent Bias Calculation


In [15]:
# Computes %Bias for a standard.
# Returns NaN if the certified value is zero (to avoid division by zero).

def percent_bias(measured: float, certified: float) -> float:
    """%Bias = (Measured - Certified) / Certified * 100."""
    if certified == 0:
        return float("nan")
    return (measured - certified) / certified * 100


#### 2. Standard QC Check
Calculates %Bias using the function above.

In [16]:
# Compares the absolute bias to a tolerance (default ±10%).
# Returns: the calculated bias "PASS" or "FAIL"

def check_standard(measured: float, certified: float, tol_pct: float = 10.0) -> dict:
    """Return status and bias for a standard."""
    bias = percent_bias(measured, certified)
    status = "PASS" if abs(bias) <= tol_pct else "FAIL"
    return {"bias_pct": bias, "status": status}


#### 3. Blank QC Check
- Computes the contamination threshold (default = 5 × DL).
- Flags the blank as "PASS" if the measured value is below the threshold.
- Flags "FAIL" if contamination is detected.

In [17]:
def check_blank(measured: float, dl: float, factor: float = 5.0) -> dict:
    """Flag contamination if measured > factor * DL."""
    threshold = factor * dl
    status = "PASS" if measured <= threshold else "FAIL"
    return {"threshold": threshold, "status": status}


In [18]:
print(check_standard(1.12, 1.10, tol_pct=10)) 
print(check_blank(0.006, dl=0.001, factor=5))  


{'bias_pct': 1.8181818181818195, 'status': 'PASS'}
{'threshold': 0.005, 'status': 'FAIL'}


#### Z‑Scores & Simple Shewhart‑Type Control Checks
When monitoring laboratory performance over time—especially for standards (Certified Reference Material: CRMs)—it’s useful to track how far each measurement deviates from the certified value.
A common way to do this is by calculating a Z‑score, which expresses deviation in units of standard deviation.

This is the foundation of `Shewhart control charts`, widely used in analytical chemistry to detect `drift, bias, and out‑of‑control` conditions.

Z‑score tells you how many standard deviations a measurement is from the expected (certified) mean:

##### z = (x – μ)/σ
Z = Measured - Mean / StDev

#### 1. Z‑score calculation
- Computes the Z‑score for a measurement.
- Returns NaN if σ = 0 (to avoid division by zero).
- This is the core metric used in control charts.


#### 2. Shewhart‑type control flag
- Applies simple Shewhart rules to classify the measurement.
- Helps identify drift or bias in the lab’s performance.
- Useful for dashboards and QC summary tables.

In [19]:
def z_score(measured: float, mu: float, sigma: float) -> float:
    if sigma == 0:
        return float("nan")
    return (measured - mu) / sigma

def control_flag(z: float) -> str:
    if abs(z) <= 2:
        return "IN CONTROL"
    elif abs(z) <= 3:
        return "WARNING (2-3σ)"
    else:
        return "OUT OF CONTROL (>3σ)"


In [20]:
z = z_score(1.20, mu=1.10, sigma=0.05)  # z = +2.0
print(z, control_flag(z))               # WARNING (2-3σ)


1.9999999999999973 IN CONTROL


#### Interval Compositing (downhole or channel) 
Interval compositing is a data-processing method used to standardize sampling intervals along a drill hole or channel.
It takes irregular raw samples (e.g., 0.7 m, 1.3 m, 2.1 m) and converts them into uniform intervals (e.g., 1 m composites).
- To make datasets consistent for geostatistics, resource modelling, and machine learning
- To reduce bias from variable sample lengths
- To align geology, assays, and geotechnical data on the same interval grid

Anything measured downhole can be composited:
- Major elements
- Trace elements
- Lithology codes
- Alteration intensities
- Density
- Geotech parameters

This Python function performs **interval compositing**, specifically length-weighted compositing, which is the standard method used in mining and exploration to combine **multiple assay intervals into a single representative value**.

We used 3 variables: (frm = start depth, to = end depth, grade = assay value (g/t, ppm, % — doesn’t matter) )

interval’s length --> length = to − from

weighted contribution of each sample -->  length × grade

For all samples --> ∑(length × grade)

composite = ∑(length × grade) / ∑length

In [21]:

from typing import List, Tuple

def length_weighted_composite(intervals: List[Tuple[float, float, float]]) -> float:
    """
    intervals: list of (from, to, grade) in g/t or ppm.
    Returns length-weighted composite grade.
    """
    total_len = 0.0
    weighted_sum = 0.0
    for frm, to, grade in intervals:
        length = to - frm
        total_len += length
        weighted_sum += length * grade
    return weighted_sum / total_len if total_len > 0 else float("nan")

# Example
intervals = [(0, 5, 0.8), (5, 10, 1.6), (10, 15, 0.4)]
print(length_weighted_composite(intervals)) 


0.9333333333333333


Interval compositing is used for:
#### ✔ Resource modelling
Standardizing intervals to 1 m, 2 m, or bench height.

#### ✔ Geostatistics
Variograms require equal‑length samples.

#### ✔ Reporting
Composites for drillhole summaries (e.g., “15 m @ 0.93 g/t Au”).

#### ✔ Multi‑element analysis
Trace elements (Bi, Te, As, Sb, Ag, etc.) are composited the same way.

#### ✔ Machine learning
Models perform better with uniform interval lengths.


### Grade × Density → Metal Content (tonnes and grams)
For a drill interval with bulk density (t/m³) and width × area.

The function calculates:

1. Volume of a rock interval
2. Tonnes of rock (using density)
3. Contained metal in grams (using grade)

This is the classic formula:

**Metal (g) = Tonnes × Grade (g/t)**

Because g/t × tonnes = grams.

Volume --> width_m × area_m2

tonnes = volume_m3 × density_tpm3 --> Density is in tonnes per cubic metre (t/m³).

grams = tonnes × grade_gpt --> Grade is in grams per tonne (g/t).



In [22]:

def metal_content(width_m: float, area_m2: float, density_tpm3: float, grade_gpt: float) -> dict:
    """
    Calculate tonnes and contained metal (grams) for a prismatic interval.
    Assumes solid rock; ignores recovery/dilution.
    """
    volume_m3 = width_m * area_m2
    tonnes = volume_m3 * density_tpm3
    grams = tonnes * grade_gpt  # g/t × t = g
    return {"volume_m3": volume_m3, "tonnes": tonnes, "metal_grams": grams}

# Example: 2 m width, 10 m2 area, 2.7 t/m3 density, 1.5 g/t Au
res = metal_content(2.0, 10.0, 2.7, 1.5)
print(res)  # ~20 m3, 54 t, 81 g Au


{'volume_m3': 20.0, 'tonnes': 54.0, 'metal_grams': 81.0}


Metal Content Usage: 

#### ✔ Resource estimation
Tonnes and metal content per block.

#### ✔ Stope design
How much metal is in a mining slice.

#### ✔ Economic evaluation
Metal value = grams × price.

#### ✔ QA/QC
Checking if composited grades make sense relative to tonnes.

#### ✔ Geometallurgy
Mass balance calculations.

## Summary
This notebook demonstrates how to implement a complete QC framework for analytical geochemistry using Python and pandas. We introduced functions for handling censored data, evaluating standards and blanks, computing Z‑scores, and generating clear QC flags. Together, these tools replicate the essential checks performed in laboratory QA/QC workflows, but in a flexible, scriptable format suitable for large datasets.

By combining these components, we can:

- Clean and standardize raw instrument outputs
- Automatically evaluate QC samples across all analytes
- Detect drift, contamination, and analytical failures
- Produce consistent QC summaries for reporting or downstream analysis

This approach ensures that any subsequent interpretation—statistical analysis, machine learning, or geochemical modelling—is built on validated, trustworthy data. The notebook serves as a practical template for anyone working with multi‑element analytical datasets who wants to bring laboratory‑grade QC into a Python environment.